In [6]:
class RedBlackTree:
    class Node:
        def __init__(self, key, color='red', parent=None, left=None, right=None):
            self.key = key
            self.color = color  # 'red' or 'black'
            self.parent = parent
            self.left = left or None
            self.right = right or None

    def __init__(self):
        self.nil = self.Node(key=None, color='black')  # NIL узел (пустой)
        self.root = self.nil

    def insert(self, key):
        new_node = self.Node(key, parent=None, left=self.nil, right=self.nil)
        parent = None
        current = self.root

        # Обычная вставка в бинарное дерево поиска
        while current != self.nil:
            parent = current
            if key < current.key:
                current = current.left
            else:
                current = current.right

        new_node.parent = parent
        if parent is None:  # Дерево пусто
            self.root = new_node
        elif key < parent.key:
            parent.left = new_node
        else:
            parent.right = new_node

        # Узел должен быть красным
        new_node.color = 'red'

        # Выполнение балансировки после вставки
        self._balance_insert(new_node)

    def _balance_insert(self, node):
        while node != self.root and node.parent.color == 'red':
            if node.parent == node.parent.parent.left:  # Родитель — левый ребенок
                uncle = node.parent.parent.right
                if uncle.color == 'red':  # Случай 1: Дядя красный
                    node.parent.color = 'black'
                    uncle.color = 'black'
                    node.parent.parent.color = 'red'
                    node = node.parent.parent
                else:
                    if node == node.parent.right:  # Случай 2: Узел — правый ребенок
                        node = node.parent
                        self._rotate_left(node)
                    # Случай 3: Узел — левый ребенок
                    node.parent.color = 'black'
                    node.parent.parent.color = 'red'
                    self._rotate_right(node.parent.parent)
            else:  # Родитель — правый ребенок
                uncle = node.parent.parent.left
                if uncle.color == 'red':  # Случай 1: Дядя красный
                    node.parent.color = 'black'
                    uncle.color = 'black'
                    node.parent.parent.color = 'red'
                    node = node.parent.parent
                else:
                    if node == node.parent.left:  # Случай 2: Узел — левый ребенок
                        node = node.parent
                        self._rotate_right(node)
                    # Случай 3: Узел — правый ребенок
                    node.parent.color = 'black'
                    node.parent.parent.color = 'red'
                    self._rotate_left(node.parent.parent)

        self.root.color = 'black'

    def delete(self, key):
        node_to_delete = self._search(self.root, key)
        if node_to_delete == self.nil:
            print(f"Узел с ключом {key} не найден.")
            return

        original_color = node_to_delete.color
        if node_to_delete.left == self.nil:
            replacement_node = node_to_delete.right
            self._transplant(node_to_delete, node_to_delete.right)
        elif node_to_delete.right == self.nil:
            replacement_node = node_to_delete.left
            self._transplant(node_to_delete, node_to_delete.left)
        else:
            successor = self._minimum(node_to_delete.right)
            original_color = successor.color
            replacement_node = successor.right
            if successor.parent == node_to_delete:
                replacement_node.parent = successor
            else:
                self._transplant(successor, successor.right)
                successor.right = node_to_delete.right
                successor.right.parent = successor

            self._transplant(node_to_delete, successor)
            successor.left = node_to_delete.left
            successor.left.parent = successor
            successor.color = node_to_delete.color

        if original_color == 'black':
            self._balance_delete(replacement_node)

    def _transplant(self, u, v):
        if u.parent is None:
            self.root = v
        elif u == u.parent.left:
            u.parent.left = v
        else:
            u.parent.right = v
        v.parent = u.parent

    def _balance_delete(self, node):
        while node != self.root and node.color == 'black':
            if node == node.parent.left:
                sibling = node.parent.right
                if sibling.color == 'red':
                    sibling.color = 'black'
                    node.parent.color = 'red'
                    self._rotate_left(node.parent)
                    sibling = node.parent.right

                if sibling.left.color == 'black' and sibling.right.color == 'black':
                    sibling.color = 'red'
                    node = node.parent
                else:
                    if sibling.right.color == 'black':
                        sibling.left.color = 'black'
                        sibling.color = 'red'
                        self._rotate_right(sibling)
                        sibling = node.parent.right

                    sibling.color = node.parent.color
                    node.parent.color = 'black'
                    sibling.right.color = 'black'
                    self._rotate_left(node.parent)
                    node = self.root
            else:
                sibling = node.parent.left
                if sibling.color == 'red':
                    sibling.color = 'black'
                    node.parent.color = 'red'
                    self._rotate_right(node.parent)
                    sibling = node.parent.left

                if sibling.right.color == 'black' and sibling.left.color == 'black':
                    sibling.color = 'red'
                    node = node.parent
                else:
                    if sibling.left.color == 'black':
                        sibling.right.color = 'black'
                        sibling.color = 'red'
                        self._rotate_left(sibling)
                        sibling = node.parent.left

                    sibling.color = node.parent.color
                    node.parent.color = 'black'
                    sibling.left.color = 'black'
                    self._rotate_right(node.parent)
                    node = self.root

        node.color = 'black'

    def _search(self, node, key):
        while node != self.nil and key != node.key:
            if key < node.key:
                node = node.left
            else:
                node = node.right
        return node

    def _minimum(self, node):
        while node.left != self.nil:
            node = node.left
        return node

    def _rotate_left(self, node):
        y = node.right
        node.right = y.left
        if y.left != self.nil:
            y.left.parent = node
        y.parent = node.parent
        if node.parent is None:
            self.root = y
        elif node == node.parent.left:
            node.parent.left = y
        else:
            node.parent.right = y
        y.left = node
        node.parent = y

    def _rotate_right(self, node):
        y = node.left
        node.left = y.right
        if y.right != self.nil:
            y.right.parent = node
        y.parent = node.parent
        if node.parent is None:
            self.root = y
        elif node == node.parent.right:
            node.parent.right = y
        else:
            node.parent.left = y
        y.right = node
        node.parent = y

    def inorder_traversal(self, node=None):
        if node is None:
            node = self.root
        if node != self.nil:
            self.inorder_traversal(node.left)
            print(node.key, f"({node.color})", end=" ")
            self.inorder_traversal(node.right)

    def print_tree(self, node=None, level=0, prefix="Root: "):
        if node is None:
            node = self.root
        if node != self.nil:
            print(" " * (level * 4) + prefix + f"{node.key} ({node.color})")
            self.print_tree(node.left, level + 1, "L--- ")
            self.print_tree(node.right, level + 1, "R--- ")

Дерево после вставки:
Root: 20 (black)
    L--- 10 (black)
        L--- 5 (red)
        R--- 15 (red)
    R--- 25 (black)
        R--- 30 (red)

Дерево после удаления 15:
Root: 20 (black)
    L--- 10 (black)
        L--- 5 (red)
    R--- 25 (black)
        R--- 30 (red)


In [4]:
tree = RedBlackTree()
tree.insert(5)
tree.insert(3)
tree.insert(6)
tree.insert(7)
tree.insert(8)
tree.insert(9)

In [5]:
print("Дерево после вставки:")
tree.print_tree()

3 (black) 5 (black) 6 (black) 7 (red) 8 (black) 9 (red) 

In [ ]:
tree.delete(15)
print("\nДерево после удаления 15:")
tree.print_tree()

In [ ]:
Сложность алгоритмов Вставка: Поиск места для вставки — 𝑂 ( log ⁡ 𝑛 ) O(logn), так как высота дерева 𝑂 ( log ⁡ 𝑛 ) O(logn). Балансировка (ротации и перекраски) — максимум 2 ротации и несколько операций перекраски, что тоже 𝑂 ( log ⁡ 𝑛 ) O(logn). Итог: 𝑂 ( log ⁡ 𝑛 ) O(logn). Ротации: Ротация выполняется за 𝑂 ( 1 ) O(1), так как она локальна и не требует обхода дерева. Пространственная сложность: Каждый узел хранит: ключ, цвет, ссылки на родителя, левого и правого ребенка. Если в дереве 𝑛 n узлов, то: 𝑂 ( 𝑛 ) O(n)

In [ ]:
Память для балансировки Память для ротаций: Ротация работает с 3-4 узлами, поэтому дополнительные затраты 𝑂 ( 1 ) O(1). Память для перекраски: Перекрашивание не требует выделения новой памяти, так как это изменение значения атрибута. Память: 𝑂 ( 1 ) O(1). Итоговые затраты на балансировку — 𝑂 ( 1 ) O(1).